# Try out framework with simulated data

In [ ]:
import glob
import os

import pandas as pd
from IPython.display import display
from ray.tune.analysis import ExperimentAnalysis

from q2_ritme.config import (
    HOST_ID,
    MLFLOW_TRACKING_URI,
    SEED_DATA,
    SEED_MODEL,
    TARGET,
    TRAIN_SIZE,
)
from q2_ritme.evaluate_models import (
    aggregate_best_models_metrics_and_configs,
    get_predictions,
    plot_best_models_comparison,
    plot_model_training_over_iterations,
    plot_rmse_over_experiments,
    plot_rmse_over_time,
    retrieve_best_models,
)
from q2_ritme.process_data import load_n_split_data
from q2_ritme.tune_models import run_all_trials

# 30.437 is avg. number of days per month
DAYS_PER_MONTH = 30.437
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
ls_model_types = ["nn", "xgb", "linreg", "rf"]
experiment_tag = "test_synthetic"

## Load and split data

In [ ]:
train_val, test = load_n_split_data(None, None, HOST_ID, TARGET, TRAIN_SIZE, SEED_DATA)

## Run all experiments

In [ ]:
result_dic = run_all_trials(
    train_val,
    TARGET,
    HOST_ID,
    SEED_DATA,
    SEED_MODEL,
    MLFLOW_TRACKING_URI,
    experiment_tag,
    model_types=ls_model_types,
    fully_reproducible=False,
)

## Evaluate best models: train_val vs. test - performance

In [ ]:
comparison_output = os.path.join("best_models", experiment_tag)

In [ ]:
best_model_dic = retrieve_best_models(result_dic)

# Assuming best_model_dic and the datasets are available
non_features = [TARGET, HOST_ID]
features = [x for x in train_val if x not in non_features]

preds_dic = {}
for model_type, tmodel in best_model_dic.items():
    train_pred = get_predictions(train_val, tmodel, TARGET, features, "train")
    test_pred = get_predictions(test, tmodel, TARGET, features, "test")
    all_pred = pd.concat([train_pred, test_pred])

    # save all predictions to model file
    path2save = os.path.join(tmodel.path, "predictions.csv")
    all_pred.to_csv(path2save, index=True)
    preds_dic[model_type] = all_pred

plot_rmse_over_experiments(preds_dic, comparison_output)

In [ ]:
plot_rmse_over_time(preds_dic, ls_model_types, DAYS_PER_MONTH, comparison_output)

## Evaluate best models: train vs. val - performance and config

In [ ]:
metrics_all, best_configs = aggregate_best_models_metrics_and_configs(result_dic)
plot_best_models_comparison(metrics_all, comparison_output)
display(best_configs)

## Evaluate one model over iterations

In [ ]:
plot_model_training_over_iterations("xgb", result_dic, labels=["data_transform"])

## Postrun evaluation over all experiments performed
experiment > trial

In [ ]:
def _min_comparison(current, best):
    return current < best


def _max_comparison(current, best):
    return current > best


def best_trial_name(analyses_ls, metric_to_evaluate, mode="min"):
    best_trial_overall = None

    if mode == "min":
        best_metric = float("inf")
        comparison_operator = _min_comparison
    else:
        best_metric = -float("inf")
        comparison_operator = _max_comparison

    for analysis in analyses_ls:
        # Get the best trial for the current analysis based on the metric
        best_trial = analysis.get_best_trial(metric_to_evaluate, mode, "all")

        # Retrieve the best metric for this trial
        best_trial_metric = best_trial.metric_analysis[metric_to_evaluate][mode]

        # Update the overall best trial if this trial has a better "trial_metric"
        if comparison_operator(best_trial_metric, best_metric):
            best_trial_overall = best_trial
            best_metric = best_trial_metric

    return best_trial_overall


def get_all_exp_analyses(experiment_dir):
    state_files = glob.glob(os.path.join(experiment_dir, "experiment_state-*.json"))
    analyses_ls = []
    for f in state_files:
        analyses_ls.append(ExperimentAnalysis(experiment_checkpoint_path=f))
    return analyses_ls

In [ ]:
# read all ExperimentAnalysis objects from this directory
best_trials_overall = {}
for model in ls_model_types:
    experiment_dir = f"best_models/{experiment_tag}/{model}"
    analyses_ls = get_all_exp_analyses(experiment_dir)

    # identify best trial from all analyses of this model type
    best_trials_overall[model] = best_trial_name(analyses_ls, "rmse_val", mode="min")

In [ ]:
best_trials_overall

## note you can retrieve config of each trial from the values, e.g.
# best_trials_overall["linreg"].config

In [ ]:
# next function would be: compare_trials(trial_id1, trial_id2)
# -> this function would load the saved train, test val predictions + the model
# config (config.json)
# TODO: create this